In [1]:
import horton
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.spatial as scs
from scipy.interpolate import CubicSpline

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
def read_input():
    lines = open('results').readlines()
    results = []
    coordinates = []
    for line in lines:
        parts = line.strip().split()
        row = {}
        if parts[0] == '#R':
            q = parts[1].strip('/').split('/')
            row['target'] = q[-2]
            if row['target'] == 'NN-2':
                row['target'] = 'N2-2'
            row['box'] = int(q[-1].split('-')[-1])
            row['energies'] = np.array([float(_) for _ in parts[2:]])
            row['numatoms'] = len(row['energies'])
            results.append(row)
        if parts[0] == '#C':
            numatoms = int(parts[1])
            row['numatoms'] = numatoms
            row['zs'] = [int(float(_)) for _ in parts[2:2+numatoms]]
            row['coords'] = np.array([float(_) for _ in parts[-3*numatoms:]]).reshape(-1, 3)
            coordinates.append(row)
    return pd.DataFrame(results), pd.DataFrame(coordinates)
r, c = read_input()
def read_energies():
    lines = open('results-energies').readlines()
    energies = []
    for line in lines:
        parts = line.strip().split()
        row = {}
        row['energy'] = float(parts[-2])
        parts = parts[0].split('/')
        row['target'] = parts[0]
        if row['target'] == 'NN-2':
            row['target'] = 'N2-2'
        row['box'] = int(parts[1].split('-')[-1])
        energies.append(row)
    return pd.DataFrame(energies)
def get_nuc_nuc_energy():
    def _get_e(row):
        e = np.outer(row.zs, row.zs)/ scs.distance.squareform(scs.distance.pdist(row.coords))
        np.fill_diagonal(e, 0)
        return e.sum(axis=0)/2    
    q = pd.merge(c, r, left_index=True, right_index=True)['coords zs target'.split()].drop_duplicates('target')
    q['enn'] = q.apply(_get_e, axis=1)
    return q['target enn zs'.split()]
    
energy_data = pd.merge(read_energies(), get_nuc_nuc_energy())

# extrapolate for infinite box size (in df as 1000)
# alchemical energies: extrapolation for 1/box size at 0 via linreg
# total energies: already converged at box site 20
rows = []
for name, group in r.sort_values('box').groupby('target'):
    extrapolated = []
    for site in range(group.numatoms.values[0]):
        es = np.array(group.energies.values.tolist())
        #plt.plot(1/group.box, es[:, site], label=name)
        try:
            extrapolated.append(np.polyfit(1/group.box, es[:, site], 1)[1])
        except:
            extrapolated.append(es[-1, site])
    row = {}
    row['box'] = 1000
    row['energies'] = extrapolated
    row['numatoms'] = len(extrapolated)
    row['target'] = name
    rows.append(row)
r = pd.concat((r,pd.DataFrame(rows)))
q = energy_data.query('box == 20').copy()
q.box = 1000
energy_data = pd.concat((energy_data, q))

/home/guido/miniconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in divide


In [3]:
targets = r[r.numatoms==12].target.unique()

In [5]:
for tidx, target in enumerate(targets):
        q = pd.merge(r[r.target.str.startswith(target)], energy_data[energy_data.target.str.startswith(target)])
        q = q[q.box == 1000]
        qq = c.iloc[r[r.target.str.startswith(target)].index[0]]

        coords = qq.coords
        zs = qq.zs
        try:
            print (target, do_mol(coords, zs))
        except:
            print (target, 'MISS')

('5-7-6-7-5-6', -114.88736934215373)
('5-7-7-5-6-6', -115.06375403902655)
('7-5-7-5-7-5', -117.41027962252605)
('5-6-6-7-6-6', -112.26455370811442)
('5-7-7-7-5-5', -117.33373980193085)
('5-7-5-7-6-6', -114.82707934496077)
('5-7-6-5-7-6', -114.82709681464456)
('5-7-6-6-7-5', -114.71188029558954)
('6-6-6-6-6-6', -109.29208625782164)


/home/guido/.local/lib/python2.7/site-packages/horton/quadprog.py:621: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  lagrange = np.linalg.lstsq(r_free.T, -g_free)[0]


('5-7-5-6-7-6', -114.84358799751243)
('5-7-6-6-6-6', -112.1427817066122)
('5-7-6-5-6-7', -114.83797887802262)
('5-6-7-6-6-6', -112.23310920136197)
('5-7-6-7-6-5', -114.75306886820667)
('5-7-5-7-7-5', -117.47885738543968)
('5-6-7-7-6-5', -114.89466516338837)
('5-7-7-6-6-5', -114.92423286443136)
('5-7-7-6-5-6', -115.06124537125811)


In [4]:
def do_mol(coords, zs):
    horton.log.set_level(0)
    mol = horton.IOData(title='test')
    mol.coordinates = np.array(coords)
    mol.numbers = np.array(zs)
    mol.pseudo_numbers = mol.numbers
    
    basisset = '6-31G'
    obasis = horton.get_gobasis(mol.coordinates, mol.numbers, basisset)
    
    olp = obasis.compute_overlap()
    kin = obasis.compute_kinetic()
    na = obasis.compute_nuclear_attraction(mol.coordinates, mol.pseudo_numbers)
    er = obasis.compute_electron_repulsion()
    
    orb_alpha = horton.Orbitals(obasis.nbasis)
    orb_beta = horton.Orbitals(obasis.nbasis)

    # Initial guess
    one = kin + na
    horton.guess_core_hamiltonian(olp, one, orb_alpha, orb_beta)
    
    grid = horton.BeckeMolGrid(mol.coordinates, mol.numbers, mol.numbers, 'fine', mode='keep', random_rotate=False)
    terms = [
        horton.UTwoIndexTerm(kin, 'kin'),
        horton.UGridGroup(obasis, grid, [
            horton.UBeckeHartree(lmax=8),
            horton.ULibXCLDA('x'),
            horton.ULibXCLDA('c_vwn'),
        ]),
        horton.UTwoIndexTerm(na, 'ne'),
    ]
    external = {'nn': horton.compute_nucnuc(mol.coordinates, mol.pseudo_numbers)}
    ham = horton.UEffHam(terms, external)
    converged = False
    for solver in (horton.CDIISSCFSolver, horton.EDIIS2SCFSolver,horton.ODASCFSolver,  horton.EDIISSCFSolver, ):
        try:
            occ_model = horton.AufbauOccModel(7, 7)
        
            occ_model.assign(orb_alpha, orb_beta)
            dm_alpha = orb_alpha.to_dm()
            dm_beta = orb_beta.to_dm()
            scf_solver = solver(1e-7, maxiter=100)
            scf_solver(ham, olp, occ_model, dm_alpha, dm_beta)
            converged = True
        except:
            continue
        if converged:
            break
    return ham.cache['energy']
#h = do_mol(coords, zs)

In [6]:
targets = r[r.numatoms!=12].target.unique()

In [7]:
for tidx, target in enumerate(targets):
        q = pd.merge(r[r.target.str.startswith(target)], energy_data[energy_data.target.str.startswith(target)])
        q = q[q.box == 1000]
        qq = c.iloc[r[r.target.str.startswith(target)].index[0]]

        coords = qq.coords
        zs = qq.zs
        try:
            print (target, do_mol(coords, zs))
        except:
            print (target, 'MISS')

('CO-1.5', -112.25762155428822)
('N2-1.7', -108.3374409872409)
('CO-1.0', -112.29253162092893)
('CO-1.7', -112.17455034438892)
('BF-0.8', -122.89870483065577)
('BF-1.7', -123.64370973838675)
('methane', -37.711208714214955)
('BF-1.0', -123.543254539585)
('N2-1.5', -108.43433510724338)
('N2-0.8', -107.97350025783689)
('CO-2', -112.07104110035469)
('HCN', -92.57897122218719)
('acetylene', -76.53790559955262)
('ethylene', -76.63078265404748)
('BF-1.2', -123.68349181639539)
('N2-2', -108.2224710958616)
('N2-1.2', -108.57627681143988)
('N2-1.0', -108.53406576353363)
('CO-1.2', -112.36176330752325)
('BF-2', -123.58589519212933)
('BF-1.5', -123.68045678379329)
('CO-0.8', -111.70980841158416)
